# Headline

In [508]:
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'output_length': 100})

In [509]:
# importing the main packages
import numpy as np
import pandas as pd
import pandas_datareader
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import os
import datetime


## Raw Data

In [510]:
# from datetime import datetime

In [511]:
from os import listdir
from os.path import isfile, join
filenames_eia = [f for f in listdir("raw_data/eia_data") if isfile(join("raw_data/eia_data", f))]
filenames_eia

['U.S._Field_Production_of_Crude_Oil_Monthly.csv',
 'U.S._Ending_Stocks_of_Total_Gasoline_Monthly.csv',
 'U.S._Exports_of_Crude_Oil_Monthly.csv',
 'U.S._Ending_Stocks_of_Crude_Oil_Monthly.csv',
 'Regular_Gasoline_Retail_Prices_Monthly.csv',
 'U.S._Imports_of_Crude_Oil_Monthly.csv']

In [512]:
filenames = [f for f in listdir("raw_data") if isfile(join("raw_data", f))]
filenames

['XOM.csv',
 'US_Inflation_Rates_Monthly.csv',
 'CPIAUCSL.csv',
 'CVX.csv',
 'SP500.csv']

In [513]:
gas_prices = pd.read_csv('raw_data/eia_data/' + 'Regular_Gasoline_Retail_Prices_Monthly.csv', skiprows=4)
gas_prices["Date"]= pd.to_datetime(gas_prices["Month"])
gas_prices.set_index('Date', inplace=True)
gas_prices.drop(columns=["Month"], inplace=True)
gas_prices.sort_index(inplace=True)
first = gas_prices.index.min()
before_first = first - datetime.timedelta(days=31) 
last = gas_prices.index[-21]


In [514]:
gas_prices = gas_prices.iloc[gas_prices.index.get_loc(gas_prices.index.min()):gas_prices.index.get_loc(last)+1]

In [515]:
stocks_gas = pd.read_csv('raw_data/eia_data/' + 'U.S._Ending_Stocks_of_Total_Gasoline_Monthly.csv', skiprows=4)
stocks_gas["Date"]= pd.to_datetime(stocks_gas["Month"])
stocks_gas.set_index('Date', inplace=True)
stocks_gas["Stocks Gas"] = stocks_gas["Series ID: PET.MGTSTUS1.M Thousand Barrels"]
stocks_gas.drop(columns=["Month","Series ID: PET.MGTSTUS1.M Thousand Barrels"], inplace=True)

In [516]:
stocks_crude = pd.read_csv('raw_data/eia_data/' + 'U.S._Ending_Stocks_of_Crude_Oil_Monthly.csv', skiprows=4)
stocks_crude["Date"]= pd.to_datetime(stocks_crude["Month"])
stocks_crude.set_index('Date', inplace=True)
stocks_crude["Stocks Crude"] = stocks_crude["Series ID: PET.MCRSTUS1.M Thousand Barrels"]
stocks_crude.drop(columns=["Month","Series ID: PET.MCRSTUS1.M Thousand Barrels"], inplace=True)


In [517]:
exports = pd.read_csv('raw_data/eia_data/' + 'U.S._Exports_of_Crude_Oil_Monthly.csv', skiprows=4)
exports["Date"]= pd.to_datetime(exports["Month"])
exports.set_index('Date', inplace=True)
exports["Exports"] = exports["Series ID: PET.MCREXUS1.M Thousand Barrels"]
exports.drop(columns=["Month","Series ID: PET.MCREXUS1.M Thousand Barrels"], inplace=True)


In [518]:
imports = pd.read_csv('raw_data/eia_data/' + 'U.S._Imports_of_Crude_Oil_Monthly.csv', skiprows=4)
imports["Date"]= pd.to_datetime(imports["Month"])
imports.set_index('Date', inplace=True)
imports["Imports"] = imports["Series ID: PET.MCRIMUS1.M Thousand Barrels"]
imports.drop(columns=["Month","Series ID: PET.MCRIMUS1.M Thousand Barrels"], inplace=True)

In [519]:
cpi = pd.read_csv('raw_data/CPIAUCSL.csv')
cpi["Date"]= pd.to_datetime(cpi["DATE"])
cpi.set_index('Date', inplace=True)
cpi.drop(columns=["DATE"], inplace=True)


In [520]:
exxon = pd.read_csv('raw_data/XOM.csv')
exxon["Date"] = pd.to_datetime(exxon["Date"])
exxon.set_index('Date', inplace=True)
exxon["Exxon"] = exxon["Close"]
exxon.drop(exxon.columns.difference(['Exxon']), 1, inplace=True)


/tmp/ipykernel_381/653263066.py:5: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [521]:
chevron = pd.read_csv('raw_data/CVX.csv')
chevron["Date"]= pd.to_datetime(chevron["Date"])
chevron.set_index('Date', inplace=True)
chevron["Chevron"] = chevron["Close"]
chevron.drop(chevron.columns.difference(['Chevron']), 1, inplace=True)


/tmp/ipykernel_381/20146316.py:5: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [522]:
sp500 = pd.read_csv('raw_data/SP500.csv')
sp500["Date"]= pd.to_datetime(sp500["Date"])
sp500.set_index('Date', inplace=True)
sp500["SP500"] = sp500["Value"]
sp500.drop(sp500.columns.difference(['SP500']), 1, inplace=True)

/tmp/ipykernel_381/2274870145.py:5: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [523]:
def cut(df):
    first = before_first
    df.sort_index(inplace=True)
    if first not in df.index:
        first = df.index.min()
    df = df.iloc[df.index.get_loc(first):df.index.get_loc(last)+1]
    return df

In [524]:
chevron = cut(chevron)
cpi = cut(cpi)
exports = cut(exports)
exxon = cut(exxon)
gas_prices = cut(gas_prices)
imports = cut(imports)
sp500 = cut(sp500)
stocks_crude = cut(stocks_crude)
stocks_gas = cut(stocks_gas)

In [525]:
df_list = [cpi, gas_prices, chevron,  exports, exxon, imports, sp500, stocks_gas]

In [526]:
df = pd.concat(df_list, axis=1)

df = df.iloc[df.index.is_month_start]

In [527]:
df_change_log = (df.pct_change()*100).apply(np.log)

In [528]:
fig = px.line(df_change_log, title='Log of percent change of variables')
fig.show()

In [529]:
df_change = (df.pct_change())


In [530]:
fig = px.line(df_change, title='Percent change of variables')
fig.show()

In [531]:
fig = px.line(df_change.drop(columns=["Exports", "Imports", "Chevron", "Exxon"]), title='Log of percent change of variables')
fig.show()

In [532]:
fig = px.line(df[["Nominal Values", "Real Values"]], title='Percent change of variables')
fig.show()

In [533]:
df.dropna()

,CPIAUCSL,Real Values,Nominal Values,Chevron,Exports,Exxon,Imports,SP500,Stocks Gas
Date,,,,,,,,,
1985-01-01,105.700,3.03,1.10,8.562500,4478.0,6.000000,84217.0,171.60,233663.0
1985-02-01,106.300,2.97,1.09,8.656250,6181.0,5.937500,59015.0,180.90,224884.0
1985-03-01,106.800,3.05,1.12,8.718750,5858.0,6.281250,86364.0,179.40,218816.0
1985-04-01,107.000,3.19,1.18,9.281250,7069.0,6.437500,102045.0,180.60,214979.0
1985-05-01,107.200,3.25,1.20,8.937500,7747.0,6.750000,115637.0,184.90,214875.0
...,...,...,...,...,...,...,...,...,...
2021-12-01,280.126,3.43,3.31,117.349998,107023.0,61.189999,199068.0,4674.77,232237.0
2022-01-01,281.933,3.41,3.31,131.330002,103760.0,75.959999,197873.0,4573.82,251753.0
2022-02-01,284.182,3.59,3.52,144.000000,92654.0,78.419998,172325.0,4435.98,250431.0


## Data clean-up

In [534]:
def data_clean(start_date, end_date, columns):
    return df

In [535]:
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble
from sklearn.ensemble import RandomForestClassifier

In [536]:
non_nan = df.dropna()

In [537]:
y = non_nan["Real Values"]
X = non_nan.drop(columns=("Real Values"))

In [538]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [539]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [540]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [541]:
rf_Model = RandomForestClassifier()

In [542]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [543]:
rf_Grid.fit(X_train, y_train)

Fitting 3 folds for each of 320 candidates, totalling 960 fits
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=17; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=17; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=17; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   0.0s

/home/luca/anaconda3/envs/project/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


960 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
960 fits failed with the following error:
Traceback (most recent call last):
  File "/home/luca/anaconda3/envs/project/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/luca/anaconda3/envs/project/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/home/luca/anaconda3/envs/project/lib/python3.9/site-packages/sklearn/ensemble/

ValueError: Unknown label type: 'continuous'

In [ ]:
print (f'Train Accuracy - : {rf_Grid.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {rf_Grid.score(X_test,y_test):.3f}')

AttributeError: 'GridSearchCV' object has no attribute 'scorer_'

## Machine Learning

### Random Forest

### Perceptron


## Results